1. split 더 잘 되게 하기
2. 재배포 금지 이런거 안뜨게 어떻게 안되나 -> 불용어로 추가?

In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


# 1. 데이터 불러오기

content/data 폴더 생성 후 training 원천 데이터 불러오기


In [ ]:
!mkdir data

In [ ]:
# 원천 training
!unzip -qq '/gdrive/MyDrive/Colab/Data/030.웹데이터 기반 한국어 말뭉치 데이터/01.데이터/1.Training/원천데이터/TS1.zip' -d /content/data

## 1-1 폴더명 변경

원천 데이터의 폴더명이 colab상에서 깨지는 문제 발생
각 폴더 안의 분류 코드를 보고 폴더명을 알맞게 변경

In [8]:
from glob import glob
import os
import json
import numpy as np
import re
from tqdm import tqdm

In [9]:
root_path = '/content/data/'
classification_code = {'SC':'IT_과학',
        'CU':'문화_패션_뷰티',
        'IN':'국제',
        'WO':'여성복지',
        'ED':'교육',
        'LC':'지역',
        'LI':'라이프스타일',
        'SP':'스포츠',
        'AC':'사건사고',
        'HE':'건강',
        'HB':'취미',
        'SG':'사회일반',
        'TL':'여행레저',
        'PO':'정치',
        'EC':'경제',
        'EN':'연예',
        'ID':'산업'}

folder_name = os.listdir(root_path)
for name in folder_name:
    os.rename(root_path+name,root_path+classification_code[os.listdir(root_path+name)[0][2:4]])


## 2. 데이터 추출

산업, 연예, 경제, 정치, 여행레저, 사회일반, 취미, 건강, 사건사고, 스포츠 안의 파일 딕셔너리 만들기

In [10]:
target_folder = ['산업', '연예', '경제', '정치', '여행레저', '사회일반', '취미', '건강', '사건사고', '스포츠']

In [11]:
files = {}
for target in target_folder:
    files[target] = glob(root_path+target+'/*')
    print(f'{target} 폴더 내 파일 수 : {len(files[target])}')

산업 폴더 내 파일 수 : 4031
연예 폴더 내 파일 수 : 4603
경제 폴더 내 파일 수 : 5072
정치 폴더 내 파일 수 : 3286
여행레저 폴더 내 파일 수 : 3162
사회일반 폴더 내 파일 수 : 3474
취미 폴더 내 파일 수 : 2746
건강 폴더 내 파일 수 : 3247
사건사고 폴더 내 파일 수 : 2813
스포츠 폴더 내 파일 수 : 2550


## 2-1 데이터 추출 함수 만들기
한 파일에서 몇개의 문장을 추출할 것인지 인자를 받아서 분야 하나당 random으로 파일을 선택하여 return하는 함수 만들기

ex) 파일당 2개 문장 추출 -> 분야당 파일 500개 random으로 선별

In [44]:
from pygments.lexers import find_lexer_class_by_name
class Preprocessor():
    def __init__(self,files:dict):
        self.files = files
        self.result_dict = {}
        self.label = {}

    def extract_file(self,field:str) -> str:
        return_dict = {}
        random_file = np.random.choice(self.files[field])
        return random_file

    def extract_news_list(self,file_path:str) -> list:
        with open(file_path,'r') as f:
            json_data = json.load(f)
        news_list = json_data['SJML']['text']
        return news_list
    
    def extract_content(self,news_list:list) -> str:
        news = np.random.choice(news_list)
        content = news['content']
        return content
        
    def extract_sentence(self,content:str) -> str:
        del_words = ['(이름) 기자']
        tmp_content = re.sub("[a-zA-Z0-9]+[@]\([\u3131-\u3163\uac00-\ud7a3]*\)","",content)
        tmp_content = re.sub("\(.{2,8}=.{2,8}\)","",tmp_content)
        tmp_content = re.sub("[\[【].{2,8}[=].{2,8}[\]】]","",tmp_content)
        tmp_content = re.sub("<.*>","",tmp_content)
        tmp_content = re.sub("[.] ",".",tmp_content)
        tmp_content = re.sub("[.]+",".",tmp_content)

        for word in del_words:
                tmp_content = tmp_content.replace(word,"")
        
        split_content = re.split("((?<=[^0-9a-zA-Z][.]))",tmp_content)
        try:
            while(True):
                split_content.remove("")
        except:
            pass

        if(len(split_content) == 0):
            return -1
        
        selected_sentence = np.random.choice(split_content)
        if(len(selected_sentence) > 10):
            return selected_sentence
        else:
            return -1

    def del_stopwords(self, selected_sentence:str) -> str:
        stopwords = ['무단전재','공감언론','재배포']
        del_special_characters = re.sub(r"[^a-zA-Z0-9\s\u3131-\u3163\uac00-\ud7a3\.]","",selected_sentence)
        result = []
        sentence_list = del_special_characters.split(" ")

        for sentence in sentence_list:
            if sentence in stopwords:
                return -1

        return ' '.join(sentence_list)
    
    def sentence_flow(self,field:str) -> str:
        file_ = self.extract_file(field)
        news_list = self.extract_news_list(file_)
        while(True):
            content = self.extract_content(news_list)
            selected_sentence = self.extract_sentence(content)
            if(selected_sentence == -1):
                continue
            clean_sentence = self.del_stopwords(selected_sentence)
            if(clean_sentence != -1):
                break

        return clean_sentence

    def extract(self) -> dict:
        for field in self.files.keys():
            self.count = 0
            result = []
            for count in tqdm(range(1000), desc=f'{field} processing'):
                clean_sentence = self.sentence_flow(field)
                if(clean_sentence[0] == '.'):
                    result.append(clean_sentence.lstrip('.'))
                else:
                    result.append(clean_sentence.lstrip())
            self.result_dict[field] = result

    def store(self):
        for field in self.result_dict.keys():
            label_list = []
            for sentence in self.result_dict[field]:
                sentence_dict = {}
                words = sentence.split()
                labeled_sentence = []
                for word in words:
                    init = ['l']*len(word)
                    init[0] = 'B'
                    labeled_sentence.append(''.join(init))
                sentence_dict['sentence'] = sentence.replace(" ","")
                sentence_dict['label'] = ''.join(labeled_sentence)
                label_list.append(sentence_dict)
            self.label[field] = label_list

        with open('/gdrive/Othercomputers/내 컴퓨터/Konkuk_ML/week6/spacing_data.txt','w') as f:
            for field in self.label.keys():
                for sentence_dict in self.label[field]:
                    sentence = ' '.join(sentence_dict['sentence'])
                    label = sentence_dict['label']
                    f.write(sentence+'\t'+label+'\n')

        print('Store Finish')


In [45]:
preprocessor = Preprocessor(files)
preprocessor.extract()

스포츠 processing: 100%|██████████| 1000/1000 [00:03<00:00, 274.19it/s]


In [46]:
preprocessor.result_dict['산업']

['향후 백화점과 대형마트 등 소매 유통을 확대해 자체 브랜드 매출을 늘릴 방침이다.',
 '해당 제품은 뉴질랜드 청정지역에서 자란 생후 6개월 이하의 양고기램 lamb를 이용해 만들었다.',
 '임팔라는 1958년 첫 출시 이래 10세대 모델까지 글로벌 시장에서 총 1600만대의 누적 판매를 기록하고 있다.',
 '해외 진출에서 선두적인 지위를 확보할 수 있을 것이라고 덧붙였다.',
 '8월 중순부터 전국 디지털프라자에서도 직접 제품을 구매할 수 있다.',
 '에센스가 함유된 미스트는 일반 미스트보다 영양이 많이 함유돼 있다.',
 'mcm은 지난 달부터 이번 시즌 컬렉션 리브 볼드의 글로벌 노마드 라이프스타일을 대변하는 11번째 쿤스트 프로젝트 전시를 하고 있으며 이름 사진전은 지난 3일 막을 내린 이름 사진전에 이은 두 번째 개인전이다.',
 '칸스톤은 석영을 주 원료로 한 엔지니어드 스톤강화 천연석으로 주방 상반 벽체 아일랜드 욕실 등에 주로 적용된다.',
 '최근 1주간 코로나19 국내 발생 이름자는 일평균 1961.4명국내 발생 1만3730명으로 직전 주일평균 2489명 대비 21.2527.6명 감소했고 수도권비수도권에서 모두 유행 규모가 줄어든 것으로 나타났다.',
 '중국에 진출한 제약바이오 업체는 gc녹십자 한미약품 sk 대웅제약 일동제약 보령제약 휴온스 바디텍메드 바이오니아 아미코젠 인트론바이오 제노포커스 등이다.',
 '이날 봉사활동에는 나세르 알 마하셔 대표와 임직원 100여명이 참여해 쪽방촌에 거주하는 독거노인과 장애인 주변 노숙자 등 500여명에게 떡국을 대접했다.',
 'family 데이는 48시간 한정 판매한다.',
 '서울 송파구 제2롯데월드 실내에서 금속물이 떨어져 사람이 다치는 사고가 발생한 가운데 서울시가 31일 긴급 안전점검을 실시한 뒤 임시사용 승인 취소 여부를 판단한다.',
 '한편 달콤커피는 매주 예비창업자들을 위한 상담 및 창업설명회를 진행하고 있으며 달콤커피 홈페이름 본사 전화를 통해서도 창업 관련 문의를 받고 있

In [47]:
preprocessor.store()

Store Finish


In [ ]:
def extract_files(extract_num:int, files:dict) -> dict:
    return_dict = {}
    file_names = files.keys()
    file_num = int(np.ceil(1000 / extract_num))
    for name in file_names:
        rand_files = np.random.choice(files[name],file_num,replace=False)
        return_dict[name] = rand_files
    return return_dict

In [ ]:
extract_num = 10
extracted_files = extract_files(extract_num,files)

## 2-2 문장 추출하는 함수 만들기
선택된 파일로부터 문장 추출하는 함수 만들기

In [ ]:
def extract_sentence(extract_num:int, extracted_files:dict) -> dict:
    return_dict = {}
    file_names = files.keys()
    # extract_num으로 나눠 떨어지면 그냥 사용
    if 1000 % extract_num == 0:
        for name in file_names:
            tmp_list = []
            for file_path in extracted_files[name]:
                print(file_path)
                print()
                with open(file_path,'r') as f:
                    json_data = json.load(f)
                selected_string_list = extract(extract_num,json_data)
                tmp_list += selected_string_list
            return_dict[name] = tmp_list
    # extract_num으로 나눠 떨어지지 않으면 마지막 파일에서 1000 % extract_num 만큼만 추출
    else:
        for name in file_names:
            tmp_list = []
            for file_path in extracted_files[name][:-1]:

                with open(file_path,'r') as f:
                    json_data = json.load(f)
                selected_string_list = extract(extract_num,json_data)
                tmp_list+=selected_string_list
            
            with open(extracted_files[name][-1],'r') as f:
                    json_data = json.load(f)
            selected_string_list = extract(1000 % extract_num,json_data)
            tmp_list+=selected_string_list
            
            return_dict[name] = tmp_list

    return return_dict

#json 파일 하나에서 무작위로 10개의 기사 선정 -> 기사 하나당 한줄의 sentence 얻어서 총 10개의 sentence를 return
def extract(extract_num:int, json_data:json) -> list:
    news = json_data['SJML']['text']
    count = 0
    select_list = []
    while(count < 10):
        random_news = np.random.choice(news,1)
        content = random_news[0]['content']
        select_string = extract_one_sentence(content)

        if(select_string == -1):
            continue
        else:
            select_list.append(select_string)         
            count += 1
            
            
    return select_list

def extract_one_sentence(content):
    stopwords = ['(이메일)','무단전재']
    count = 0
    print("content: ",content)
    tmp_content = re.sub("[.] ",".",content)
    tmp_content = re.sub("[.]+",".",tmp_content)
    tmp_content = re.sub("<.*>","",tmp_content)
    tmp_content = re.sub("[a-zA-Z]+@(.*)","",tmp_content)
    
    split_content = re.split("((?<=[^0-9a-zA-Z][.]))",tmp_content)
    print("2",split_content)
    ## 함수 변경해야함
    while(True):
        select = np.random.choice(split_content)
        for word in stopwords:
            if select.find(word) != -1:
                return -1
        if(len(select) > 2):
            break
    print("3",select)
    return select



In [ ]:
with open('/content/data/사회일반/BWSG217000051460.json','r') as f:
    json_data = json.load(f)
print(json_data['SJML']['text'][1])

{'title': '부천시, 지역사회 통합돌봄 분야 ‘전국 최고’', 'subtitle': '', 'content': "부천시가 지역사회 통합돌봄 분야 전국 최고임을 입증했다.. 시는 지난 18일 보건복지부 주관으로 개최한 2020년 복지행정상 지역사회 통합돌봄 서비스 제공 분야에서 '대상'을 차지했다고 21일 밝혔다.. '지역사회 통합돌봄 서비스는 돌봄이 필요한 주민(노인, 장애인 등)들이 살던 곳에서 개개인의 욕구에 맞는 서비스를 누리고, 지역사회와 함께 어울려 살아갈 수 있도록 주거, 보건의료, 요양, 돌봄, 일상생활의 지원이 통합적으로 확보되는 지역 주도형 사회서비스 정책이다.. 시상식은 코로나19 예방을 위해 정부세종컨벤션센터와 각 지자체를 이원 영상으로 연결한 비대면 방식으로 진행됐다.. 부천시는 장덕천 부천시장과 담당 부서 일부 인원이 방역 수칙을 준수하며 비대면 방식으로 트로피 전달, 수상대표 소감, 릴레이 노래 등에 참여했다.. 복지행(이름) 지자체의 복지 수준 제고와 국정과제 연계 강화를 위해 보건복지부에서 매년 지자체를 대상으로 실시하는 평가다. 지역사회 통합돌봄 분야는 올해 주요 정책 분야로 신설됐다.. 시는 지역사회 통합돌봄 서비스 제공 분야에서 ▲초고령사회 대비 '모두가 누리는 지역사회 통합돌봄 모형 개발'을 위한 사업계획 수립 ▲행정복지센터 중심의 통합돌봄 전달체계 기반 마련 ▲민·관 협력 네트워크 구축을 통한 지역케어회의 및 통합돌봄 협의체 구성·운영 ▲병원·시설 퇴원환자 통합돌봄 서비스 연계 실적 등을 인정받아 대상의 영예를 안았다.. 특히 통합돌봄 전담팀을 (이름)고 10개 종합사회복지관과 작은 보건소인 100세 건강실을 1대 1로 매칭하며 통합돌봄 전달체계 구축에 힘쓴 점에서 호평을 받았다.. 통합돌봄 대상자는 소득 기준이 아닌 돌봄 필요도에 따라 발굴되며, 다직종·다기관 전문가가 참여한 지역케어회의에서 대상자 중심의 서비스를 지원한다. 시에서 실시한 효과성 분석에 따르면 이 같은 통합돌봄 서비스가 삶의 질, 사회적 안정감 등

In [ ]:
for i in extracted_sentence.keys():
    print(len(extracted_sentence[i]))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
